In [1]:
import onnx
import copy

In [16]:
flow = onnx.load("evits_onnx/flow.onnx")
dec = onnx.load("evits_onnx/dec.onnx")

In [3]:
len(flow.graph.node)

533

In [4]:
len(dec.graph.node)

200

In [17]:
mul = onnx.helper.make_node(op_type='Mul', inputs=['z', 'y_mask'], outputs=['z_out'], name="Mul_out")
flow.graph.node.append(mul)

for node in dec.graph.node:
    for i in range(len(node.input)):
        node.input[i] = "dec_" + node.input[i]
    for i in range(len(node.output)):
        node.output[i] = "dec_" + node.output[i]
    node.name = "dec_" + node.name
    flow.graph.node.append(node)

for weight in dec.graph.initializer:
    weight.name = "dec_" + weight.name
    flow.graph.initializer.append(weight)

_ = flow.graph.output.pop()

flow.graph.output.append(dec.graph.output[0])
flow.graph.output[0].name = "dec_o"
flow.graph.node[534].input[0] = "z_out"


In [18]:
onnx.save(flow, "flow_dec.onnx")